In [2]:
from os import listdir, walk
from os.path import isfile, join, basename

# import xml.etree.ElementTree as etree

import re

import gensim
# from gensim.models.word2vec import word2vec


from collections import Counter, defaultdict

In [3]:
# print all the variable names at the end of a cell ( not just the last one)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use("TkAgg")
%matplotlib inline

import progressbar

In [5]:
from keras.preprocessing.text import text_to_word_sequence

/storage/xalex/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Build a corpus of Romanian words from Romanian ebooks converted to text files.

Steps:
1. read all input files ( generated by WikiExtractorX from wiki dumps) 
2. do some cleaning
3. build list of words with counters 

Text files are in "../data/books_txt/*"

In [6]:
DATA_PATH = "../data/"
TEXT_FILES_PATH = join(DATA_PATH, "books_txt")
DOC_FILES_PATH = join(DATA_PATH, "books_doc")
OUT_FILES_PATH = join(DATA_PATH, "books_out")

In [7]:
number_of_files = 0
for (root_dir, dirs, files) in walk(DOC_FILES_PATH):
    number_of_files += len(files)
print(f"number of files to process: {number_of_files}")

number of files to process: 8836


In [21]:
# convert all DOC files in data/books_doc into txt files in data/books_txt

import textract

files_counter = 0
files_to_process = []
for (root_dir, dirs, files)  in walk(DOC_FILES_PATH):
    for local_file_name in files:
        files_counter += 1 
        local_file_path = join(root_dir, local_file_name)
        if isfile(local_file_path):
            files_to_process.append(local_file_path)
            
files_to_process = sorted(files_to_process)
print(f"number of files to process: {len(files_to_process)}")
number_of_files = len(files_to_process)
# for i, b in enumerate(files_to_process): 
#     print(i, b)

number of files to process: 8835


In [24]:
def convert_doc_to_txt(list_files):

    bar = progressbar.ProgressBar()
    error_files = []
    for local_file_path in bar(list_files):
        try:
            file_content = textract.process(local_file_path)
            with open(join(OUT_FILES_PATH, basename(local_file_path)+".txt"), "wb") as out_file:
                out_file.write(file_content)
    #         print(f"ok: {local_file_path}")
        except UnicodeDecodeError:
            print(f"error reading {local_file_path}")
            error_files.append(local_file_path)

In [25]:
convert_doc_to_txt(files_to_process[:10])

100% (10 of 10) |#########################| Elapsed Time: 0:00:19 Time: 0:00:19


In [8]:

files_to_process = []

files_counter = 0
for (root_dir, dirs, files) in walk(OUT_FILES_PATH):
    files_counter += len(files)

files_counter = 0
for (root_dir, dirs, files)  in walk(OUT_FILES_PATH):
    for local_file_name in files:
        files_counter += 1 
        local_file_path = join(root_dir, local_file_name)
        if isfile(local_file_path):
            files_to_process.append(local_file_path)
            

In [11]:
# function to read TXT files in list_files and build a counter of words

def build_word_counters(list_files):

    word_counter = Counter()
    word_counter_lower = Counter()
    
    bar = progressbar.ProgressBar()
    for local_file_path in bar(list_files):
        try:
            with open(local_file_path, 'r', ) as text_file:
                for word in text_to_word_sequence(text_file.read(), 
                                                    filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n',
                                                    lower=True):
                    word_counter_lower[word] += 1
            # case insensitive
            with open(local_file_path, 'r', encoding='utf8') as text_file:
                for word in text_to_word_sequence(text_file.read(), 
                                                    filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n',
                                                    lower=False):
                    word_counter[word] += 1
        except UnicodeDecodeError:
            print(f"error reading {local_file_path}")
            
    return word_counter, word_counter_lower
        

In [13]:

word_counter, word_counter_lower = build_word_counters(files_to_process[:10000])


print(f"Total number of words in corpus: {sum(word_counter_lower.values())}")
print(f"{len(word_counter)} distinct case sensitive words")
print(f"{len(word_counter_lower)} distinct case INsensitive words")

words_by_len = Counter()
for w in word_counter_lower:
    words_by_len[len(w)] += 1
    
for i in range(max(words_by_len.keys())):
    print(f"{i}: {words_by_len[i]}")

100% (8835 of 8835) |#####################| Elapsed Time: 0:21:28 Time: 0:21:28


Total number of words in corpus: 573841112
4767401 distinct case sensitive words
4166617 distinct case INsensitive words
0: 0
1: 345
2: 4337
3: 43324
4: 141496
5: 281783
6: 414840
7: 525973
8: 538411
9: 538024
10: 451566
11: 362760
12: 271680
13: 193163
14: 131882
15: 87478
16: 57287
17: 38364
18: 25412
19: 17442
20: 11746
21: 8036
22: 5418
23: 3908
24: 2587
25: 1798
26: 1369
27: 1028
28: 777
29: 625
30: 488
31: 433
32: 327
33: 289
34: 283
35: 195
36: 182
37: 181
38: 130
39: 119
40: 128
41: 97
42: 80
43: 67
44: 73
45: 47
46: 60
47: 46
48: 42
49: 32
50: 38
51: 35
52: 26
53: 26
54: 21
55: 11
56: 14
57: 17
58: 15
59: 27
60: 11
61: 14
62: 8
63: 11
64: 12
65: 15
66: 11
67: 6
68: 11
69: 6
70: 5
71: 7
72: 12
73: 6
74: 9
75: 9


In [16]:
import pickle

with open(join(DATA_PATH, "corpus_books_counters_v1.pickle"), "wb") as pickle_file:
    pickle.dump([word_counter, word_counter_lower], pickle_file)

In [18]:
for w in word_counter_lower:
    if len(w)> 40:
        pass
#         print(w)

In [19]:
import re
import unicodedata as UD
    
# TEXT_FILES_PATH = "../data/text"
word_counter = Counter()
word_counter_lower = Counter()
files_to_process = []

# build a list of all files to process
files_counter = 0
for (root_dir, dirs, files)  in walk(OUT_FILES_PATH):
    for local_file_name in files:
        files_counter += 1 
        local_file_path = join(root_dir, local_file_name)
        if isfile(local_file_path):
            files_to_process.append(local_file_path)
            







In [20]:
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/xalex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [124]:
re_sentences = re.compile("[.!?][\n ]")

res= re_sentences.split("""
înţelege mai bine ceea ce am în vedere.)

In 1974, când se aniversau 150 de

ani de la naşterea filosofului, am scris un scurt articol despre conceptul

judecăţii analitice la Kant.
""".replace("\n\n", " "))

res
strip_chars = chr(9)+chr(32)+"\n"
for l in " 	Acum mai bin".lstrip(strip_chars):
    print(f"{l}: {ord(l)}")

['\nînţelege mai bine ceea ce am în vedere.) In 1974, când se aniversau 150 de ani de la naşterea filosofului, am scris un scurt articol despre conceptul judecăţii analitice la Kant',
 '']

A: 65
c: 99
u: 117
m: 109
 : 32
m: 109
a: 97
i: 105
 : 32
b: 98
i: 105
n: 110


In [89]:
re_strip = re.compile("[„”<>–«»;//“’'_\"\u200b\u00ad]")
s = "asdfasdf" + chr(8203) + "aaaa" + chr(173) + "bbb"
s
re_strip.sub("", s)
UD.name("\u00ad")
UD.name(chr(173))
UD.name("\u200b")
try:
    print(UD.name("\uFDD3"))
except ValueError:
    print("err")
hex(173)

'asdfasdf\u200baaaa\xadbbb'

'asdfasdfaaaabbb'

'SOFT HYPHEN'

'SOFT HYPHEN'

'ZERO WIDTH SPACE'

err


'0xad'

In [118]:
for l in "ÎĂȘŞȚŢîășşțţîâ":
    print(f"{l}: {UD.name(l)}")
re_contains_diacriticals = re.compile('[ÎĂȘŞȚŢîășşțţîâ]')
                                     
sentence = "Simţea razele soarelui ca fiind mai primitoare de data aceasta."

if re_contains_diacriticals.search(sentence):
    print("found")
UD.lookup('LATIN CAPITAL LETTER T WITH CEDILLA')

Î: LATIN CAPITAL LETTER I WITH CIRCUMFLEX
Ă: LATIN CAPITAL LETTER A WITH BREVE
Ș: LATIN CAPITAL LETTER S WITH COMMA BELOW
Ş: LATIN CAPITAL LETTER S WITH CEDILLA
Ț: LATIN CAPITAL LETTER T WITH COMMA BELOW
Ţ: LATIN CAPITAL LETTER T WITH CEDILLA
î: LATIN SMALL LETTER I WITH CIRCUMFLEX
ă: LATIN SMALL LETTER A WITH BREVE
ș: LATIN SMALL LETTER S WITH COMMA BELOW
ş: LATIN SMALL LETTER S WITH CEDILLA
ț: LATIN SMALL LETTER T WITH COMMA BELOW
ţ: LATIN SMALL LETTER T WITH CEDILLA
î: LATIN SMALL LETTER I WITH CIRCUMFLEX
â: LATIN SMALL LETTER A WITH CIRCUMFLEX
found


'Ţ'

In [129]:
from nltk import sent_tokenize
import nltk
#nltk.download('punkt')

def process_files_better(list_files):
    
    word_counter = Counter()
    word_counter_lower = Counter()
    
    file_stats = {}
    
    # use this re to check if paragraphs use propper spelling
    re_contains_diacriticals = re.compile('[ÎĂȘŞȚŢîășşțţîâ]')
    # the "space" in re_contains_weird_chars is not regular 'space', but '\xa0' - non braking space
    re_contains_weird_chars = re.compile('[^a-zA-Z0-9,.?! ÎĂȘȚîășțîâ\:\-\(\)]')
    # find URLs
    re_url = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    #these chars will be stripped from text
    """
    8203 BN ZERO WIDTH SPACE
    173 BN SOFT HYPHEN
    \u0627-\u064a - arabic
    \u0590-\u05FF - hebrew
    """
    re_strip = re.compile("[„”<>–«»;//“’'_\"\u200b\u00ad\u0627-\u064a\u0590-\u05FF]")

    #re_contains_right_to_left = re.compile('\u200f')
    counter_weird_chars = Counter()
    count_sentences_no_diacriticals = 0 
    count_sentences = 0

    sentences_file = open(join(DATA_PATH, "sentences"), "w", encoding='utf8')
    sentences_skipped_file = open(join(DATA_PATH, "sentences_skipped"), "w", encoding='utf8')

    strip_chars = chr(9)+chr(32)+"\n"
    re_sentences = re.compile("[.!?]([\n ])")
    
    bar = progressbar.ProgressBar()
    index = -1
    for local_file_path in bar(list_files):
        index += 1
        char_class_counter = Counter()
        
        with open(local_file_path, 'r', encoding='utf8') as text_file:
            paragraphs = text_file.read().split("\n\n")

        sentences = []
        for p in paragraphs: 
            sentences.extend(re_sentences.split(p))
#         for line in lines:
#             sentences.extend(sent_tokenize(re_url.sub('',line.replace("\n\n", " "))))
        
        sentences = [re_strip.sub('',x.replace(u'\xa0', ' ').rstrip(strip_chars).lstrip(strip_chars).replace("\n", " ")) for x in sentences]
        #print(sentences)
        for sentence in sentences:
            skip = False
            skip_char_class = ""
            skip_diacriticals = ""
            skip_weird_chars = ""
            
            if len(sentence) <3:
                continue
            count_sentences += 1
            
    #         print(f"--{sentence}--")
            
            for l in sentence:
                ud_class = UD.bidirectional(l)
    #             if UD.bidirectional(l) not in ['L', 'CS', 'EN','ES','ON','ET','WS','R','AL']:
                # skip sentences with RIGHT-TO-LEFT characters
                
                if ud_class not in ['L', 'CS', 'EN','ES','ON','ET','WS']:
                    char_class_counter[ud_class] +=1
                    skip = True
                    skip_char_class = "char_class"
                    try:
                        ud_name = UD.name(l)
                    except ValueError:
                        ud_name = "undefined"
                    print(f"->{l}<- {ord(l)} {ud_class} {ud_name}")
#                     print(f"->{l}<- {ord(l)} ")
        
            if not re_contains_diacriticals.search(sentence):
                count_sentences_no_diacriticals += 1
                skip = True
                skip_diacriticals = "diacriticals"

            if False:
                res = re_contains_weird_chars.findall(sentence)
                skip_weird_chars = "weird_chars"
                for c in res:
        #            print(f"<{c}>, ascii={ord(c)}")
                    counter_weird_chars[c] +=1
        #             print(sentence, end="\n---------\n")
                    #skip = True

            
            if skip:
                sentences_skipped_file.write(f"{skip_char_class} {skip_diacriticals}: {sentence}.\n+")
                continue
            else:
                sentences_file.write(sentence+".\n")
                # case sensitive list of words
                for word in text_to_word_sequence(sentence, 
                                            filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n',
                                            lower=True):
                    word_counter_lower[word] += 1
                # case insensitive list of words
                for word in text_to_word_sequence(sentence, 
                                            filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n',
                                            lower=False):
                    word_counter[word] += 1
        
        if len(char_class_counter):
            print(f"{i}: {basename(local_file_path)}")
            for cls, cnt in char_class_counter.items():
                print(f"{cls}: {cnt}")    
            
    sentences_file.close()
    sentences_skipped_file.close()
    
    
    print(f"total sentences: {count_sentences}")
    print(f"sentences without diacriticals: {count_sentences_no_diacriticals}")
    
    for c in counter_weird_chars:
        print(f"<{c}> -> {counter_weird_chars[c]} {ord(c)}")
    
    return word_counter, word_counter_lower




In [132]:
files_to_process = sorted(files_to_process)
#count_w, count_w_l = process_files_better([files_to_process[i] for i in [4703 ,5034 ,5196 ,5649 ,5906 ,5866 ,5906] ])
files_to_process[1300]
count_w, count_w_l = process_files_better(files_to_process)

print(f"{len(count_w)} case sensitive words")
print(f"{len(count_w_l)} case INsensitive words")

'../data/books_out/Anonim-Povestiri_V3_08__..doc.txt'

 11% (1010 of 8835) |##                   | Elapsed Time: 0:03:13 ETA:  0:24:21

->٬<- 1644 AN ARABIC THOUSANDS SEPARATOR
75: Angela_Ionescu-Psihoterapie,notiuni_Introductive_10__.doc.txt
AN: 1


 12% (1065 of 8835) |##                   | Elapsed Time: 0:03:33 ETA:  0:43:31

->̕<- 789 NSM COMBINING COMMA ABOVE RIGHT
75: Anonim-1001_De_Nopti_V3_2.0_09__.doc.txt
NSM: 1


...

->﷓<- 64979  undefined
->﷓<- 64979  undefined
->﷓<- 64979  undefined
->﷓<- 64979  undefined
75: Marcel_Luca-Atac_Cu_Spitfire_La_Bunderdaal_1.0_10__.doc.txt
: 4
->﷓<- 64979  undefined
75: Marcel_Luca-Ghicitorul_In_Tarot_10__.doc.txt
: 1


 90% (8026 of 8835) |###################  | Elapsed Time: 0:32:39 ETA:  0:03:15

->؛<- 1563 AL ARABIC SEMICOLON
75: Panait_Istrati-In_Lumea_Mediteranei_0.99_09__.doc.txt
AL: 1


100% (8835 of 8835) |#####################| Elapsed Time: 0:36:08 Time: 0:36:08


total sentences: 38171516
sentences without diacriticals: 4094316
4011212 case sensitive words
3505156 case INsensitive words


In [137]:
import pickle

with open(join(DATA_PATH, "corpus_books_counters_v2.pickle"), "wb") as pickle_file:
    pickle.dump([count_w, count_w_l], pickle_file)

In [136]:
for w,c in count_w.most_common(100):
    print(f"{w}: {c}")

de: 20467435
şi: 15459598
în: 12399436
să: 9781549
cu: 7243189
se: 7160924
la: 7097814
pe: 7074652
o: 7035067
a: 6712451
nu: 5722142
că: 5685651
care: 5507265
mai: 5024854
—: 4948695
din: 4659080
un: 4599559
ce: 3239848
ca: 2973931
lui: 2953495
pentru: 2566218
fi: 2509770
era: 2037347
este: 1808648
el: 1802563
dar: 1533690
când: 1532227
am: 1485236
ei: 1477487
al: 1476197
e: 1389217
fost: 1334515
îi: 1323791
Nu: 1247923
sau: 1236420
cum: 1220197
mă: 1198362
nici: 1153310
sunt: 1109576
Şi: 1095925
În: 1093985
ea: 1089049
ar: 1075523
prin: 1068573
după: 1055051
au: 1045534
dacă: 1042440
îl: 1016000
va: 981733
îşi: 966931
decât: 954169
le: 947878
aşa: 908107
Dar: 892769
iar: 867403
mult: 863903
fără: 833792
lor: 831796
s-a: 816545
tot: 809890
spre: 803448
până: 773387
bine: 761753
poate: 758675
ne: 749806
asta: 743407
fie: 732998
putea: 731206
toate: 730230
te: 726933
ale: 718256
timp: 716140
avea: 707502
eu: 692302
atât: 682092
ai: 667469
despre: 663060
chiar: 659000
De: 638154
mare: 625

In [139]:
print(max([len(word) for word, count in count_w_l.items()]))
#print([word for word, count in word_counter_lower.items() if len(word)>1600])

76


In [149]:
deleted_words = []

counter_word_length = Counter()
for w in count_w_l:
    counter_word_length[len(w)] += 1
    if len(w) >= 27:
        deleted_words.append(w)
#         print(f"{w} are {len(w)} chars")
        pass

for w in deleted_words:
   del count_w_l[w]
        
for w,c in sorted(counter_word_length.items()):
    print(f"{w}: {c} ")

1: 322 
2: 3473 
3: 33277 
4: 115300 
5: 239071 
6: 353394 
7: 440290 
8: 453281 
9: 449555 
10: 373768 
11: 299294 
12: 225260 
13: 162066 
14: 112089 
15: 76356 
16: 51530 
17: 35288 
18: 23975 
19: 16576 
20: 11524 
21: 7896 
22: 5477 
23: 3831 
24: 2621 
25: 1839 
26: 1402 


In [150]:
# this is the input I would like to correct

input_text = "Ambasadorul SUA la Natiunile Unite, Nikky Haley, a declarat miercuri ca Statele Unite considera ca Rusia este responsabila pentru atacul chimic din Marea Britanie asupra fostului spion rus si a fiicei sale. De asemenea, Consiliul de Securitate al ONU ar trebui sa ia masuri, mai spune aceasta, potrivit reuters.com.Premierul britaqnic Theresa May a anuntat, intr-un discurs tinut in parlament, ca va expulza 23 de diplomati rusi. May a adaugat ca este cel mai masiv val de expulzari din ultimii peste 30 de ani. Ambasadorul Moscovei la Londra a avertizat ca vor exista masuri reciproce daca Regatul Unit expulzeaza diplomati rusi."

In [151]:
print(input_text)

Ambasadorul SUA la Natiunile Unite, Nikky Haley, a declarat miercuri ca Statele Unite considera ca Rusia este responsabila pentru atacul chimic din Marea Britanie asupra fostului spion rus si a fiicei sale. De asemenea, Consiliul de Securitate al ONU ar trebui sa ia masuri, mai spune aceasta, potrivit reuters.com.Premierul britaqnic Theresa May a anuntat, intr-un discurs tinut in parlament, ca va expulza 23 de diplomati rusi. May a adaugat ca este cel mai masiv val de expulzari din ultimii peste 30 de ani. Ambasadorul Moscovei la Londra a avertizat ca vor exista masuri reciproce daca Regatul Unit expulzeaza diplomati rusi.


In [155]:
count_w['acasă']

152299

In [33]:
# save word_counter and word_counter_lower to picke files ( because they are expensive to generate)

import pickle

FILE_FOR_COUNTERS = join(DATA_PATH,'saved_counters.pickle')

In [35]:
with open(FILE_FOR_COUNTERS, 'wb') as dump_file:
    pickle.dump([word_counter, word_counter_lower], dump_file)

In [242]:
# ( in case we need to) restore word_counter and word_counter_lower from files 
with open(FILE_FOR_COUNTERS, 'rb') as dump_file:
    word_counter, word_counter_lower = pickle.load(dump_file)

In [36]:
# read cleaned counter
with open('../data/pickle_clean_words_lower.pickle', 'rb') as dump_file:
    word_counter_lower = pickle.load(dump_file)

In [156]:
def generate_misspellings(w):
    """Generate all miss-spellings of word w caused by missing diacritical marks"""
    
    specials = {'î':'i', 'ă':'a', 'ș':'s', 'ş':'s', 'ț':'t', 'ţ':'t', 'î':'i', 'â':'a'}
    specials = {'î':'i', 'ă':'a', 'ș':'s', 'ş':'s', 'ț':'t', 'ţ':'t', 'î':'i', 'â':'a', '-':''}

    """ÎĂȘŞȚŢîășşțţîâ"""
    
    spellings = [""]
    count_specials_found=0
    pos=0
    for l in w:
#         print(l)
        pos += 1
        if l in specials:
            count_specials_found +=1
            if count_specials_found > 5:
                print(f"{w}: many specials: {count_specials_found}")
            if count_specials_found == 10:
                print(f"{w}: TOO many specials: {count_specials_found}")
                return spellings
            spellings = [spelling + l for spelling in spellings]
#             print(spellings)
#             print(f"adding: {[ tmp_s[0:-1] + specials[l] for tmp_s in spellings]}")
            spellings.extend( [ tmp_s[0:-1] + specials[l] for tmp_s in spellings])
#             print(f"new spelling {w[0:pos]}")
        else:
            spellings = [spelling + l for spelling in spellings]
#         print(spellings)
#         print()
    return spellings

In [157]:
# generate_misspellings("îpârâu")
generate_misspellings("într-oî")
generate_misspellings("încăpățânată")

['într-oî',
 'intr-oî',
 'întroî',
 'introî',
 'într-oi',
 'intr-oi',
 'întroi',
 'introi']

încăpățânată: many specials: 6


['încăpățânată',
 'incăpățânată',
 'încapățânată',
 'incapățânată',
 'încăpațânată',
 'incăpațânată',
 'încapațânată',
 'incapațânată',
 'încăpătânată',
 'incăpătânată',
 'încapătânată',
 'incapătânată',
 'încăpatânată',
 'incăpatânată',
 'încapatânată',
 'incapatânată',
 'încăpățanată',
 'incăpățanată',
 'încapățanată',
 'incapățanată',
 'încăpațanată',
 'incăpațanată',
 'încapațanată',
 'incapațanată',
 'încăpătanată',
 'incăpătanată',
 'încapătanată',
 'incapătanată',
 'încăpatanată',
 'incăpatanată',
 'încapatanată',
 'incapatanată',
 'încăpățânata',
 'incăpățânata',
 'încapățânata',
 'incapățânata',
 'încăpațânata',
 'incăpațânata',
 'încapațânata',
 'incapațânata',
 'încăpătânata',
 'incăpătânata',
 'încapătânata',
 'incapătânata',
 'încăpatânata',
 'incăpatânata',
 'încapatânata',
 'incapatânata',
 'încăpățanata',
 'incăpățanata',
 'încapățanata',
 'incapățanata',
 'încăpațanata',
 'incăpațanata',
 'încapațanata',
 'incapațanata',
 'încăpătanata',
 'incăpătanata',
 'încapătanata

In [161]:
spellings_lower = defaultdict(Counter)

for word, counter in count_w_l.items():
    misspellings = generate_misspellings(word)
#     print(f"{word}: {len(misspellings)}")
#     print(misspellings)
    for misspelling in misspellings:
        spellings_lower[misspelling][word] = counter
#     break

încăpăţânată: many specials: 6
încăpăţânaţi: many specials: 6
încăpăţâna-re: many specials: 6
lasă-mă-să-te-las: many specials: 6
lasă-mă-să-te-las: many specials: 7
stră-stră-străbunică: many specials: 6
încăpăţână: many specials: 6
a-mai-prima-în-poiată: many specials: 6
nu-ştiu-câta-oară: many specials: 6
împrăştiaţi-vă: many specials: 6
împărţiţi-vă: many specials: 6
slani-fără-cârlionţi: many specials: 6
înfăşurându-şi: many specials: 6
îmbrăţişându-mă: many specials: 6
îmbrăţişându-mă: many specials: 7
înălţându-şi: many specials: 6
înfăţişându-se: many specials: 6
stră-stră-stră-slrăbunicul: many specials: 6
stră-stră-stră-slrăbunicul: many specials: 7
mcăpăţânându-se: many specials: 6
încă-oăţânasem: many specials: 6
în-căpăţânare: many specials: 6
îm-răţişându-mi: many specials: 6
îm-răţişându-mi: many specials: 7
înfăţăşându-i: many specials: 6
înfăţăşându-i: many specials: 7
încăpăţându-mă: many specials: 6
încăpăţându-mă: many specials: 7
îmbrăţişându-şi: many specials: 6
î

In [162]:
spellings_lower['acasa']
spellings_lower['parau']
spellings_lower['intro']
spellings_lower['incapatanata']

Counter({'-acasă': 8,
         'a-c-a-s-ă': 1,
         'a-ca-să': 1,
         'a-cas-a': 1,
         'a-casă': 11,
         'aca-sa': 2,
         'aca-să': 10,
         'acas-a': 1,
         'acasa': 44,
         'acasâ': 5,
         'acasă': 156899,
         'acasă-': 7,
         'acaşa': 1})

Counter({'parau': 43,
         'parâu': 2,
         'parău': 1,
         'pâ-râu': 3,
         'pârau': 28,
         'pârâu': 3683,
         'pârău': 239,
         'pârău-': 1,
         'părâu': 17,
         'pără-u': 1,
         'părău': 62})

Counter({'-într-o': 33,
         '-întro': 1,
         'in-tr-o': 23,
         'intr-o': 796,
         'intro': 215,
         'intro-': 22,
         'inţr-o': 3,
         'î-ntr-o': 4,
         'în-tr-o': 219,
         'în-tro': 1,
         'în-ţr-o': 3,
         'înt-r-o': 1,
         'într--o': 1,
         'într-o': 556906,
         'într-o-': 7,
         'întro': 1812,
         'întro-': 1,
         'înţr-o': 20})

Counter({'încă-păţânată': 2, 'încăpăţânata': 135, 'încăpăţânată': 2590})

In [164]:
# save spellings_lower to pickle file so we ca use it later

# DATA_PATH = "../data/"
with open(join(DATA_PATH,'books_spellings_v2.pickle'), 'wb') as pickle_file:
    pickle.dump( spellings_lower, pickle_file)
    
len(spellings_lower)

7618490

In [41]:
word_counter_lower["unit"]

7039

In [42]:
spellings_lower['acasa']
spellings_lower['parau']
spellings_lower['intro']

Counter({'acasa': 113, 'acasă': 3934})

Counter({'parau': 20,
         'pârau': 2,
         'pârâu': 255,
         'pârău': 13,
         'părau': 1,
         'părâu': 1,
         'părău': 55})

Counter({'in-tro': 1,
         'intr-o': 577,
         'intro': 63,
         'în-tro': 4,
         'într-o': 33569,
         'întro': 33})

In [43]:
len(spellings_lower)

1367905

In [44]:
max([len(word) for word in spellings_lower])

29

In [45]:
for word in spellings_lower:
    if len(word) == 29:
        print(word)

apomnemomneumata-memorabilele
nowikinowikirăpitnowikinowiki
nowikinowikirapitnowikinowiki
kurfürstlich-brandenburgische
raionuluimunicipiuluiunității
raionuluimunicipiuluiunitații
raionuluimunicipiuluiunitătii
raionuluimunicipiuluiunitatii
romulusbărbulescugeorgeanania
romulusbarbulescugeorgeanania
richmondfredericksburgpotomac
perhidrociclopentanofenantren
monarhiști-constituționaliști
monarhisti-constituționaliști
monarhiști-constitutionaliști
monarhisti-constitutionaliști
monarhiști-constituționalisti
monarhisti-constituționalisti
monarhiști-constitutionalisti
monarhisti-constitutionalisti
fessanvilliers-mattanvilliers
ampaacrealaudiorealvideompeg4
autotohtonist-tradiționalistă
autotohtonist-traditionalistă
autotohtonist-tradiționalista
autotohtonist-traditionalista
continuitateadiscontinuitatea
ftaliltetrahidroizochinolinic
nominativacuzativdativgenitiv
bucureștioradeabudapestaviena
bucurestioradeabudapestaviena
strigoniensis-budapestinensis
pestszentlőrinc-pestszentimre
bezbednost

In [165]:
for input_word in text_to_word_sequence(input_text.lower(), 
                                        filters='!"#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n',
                                        lower=False):
    print(f"{input_word}: {word_counter_lower[input_word]} {word_counter_lower[input_word.lower()]} {spellings_lower[input_word]}")

ambasadorul: 0 0 Counter({'ambasadorul': 4841, '-ambasadorul': 2, 'ambasa-dorul': 1})
sua: 0 0 Counter({'sua': 7034, 'şua': 30, 'suă': 8, 'suâ': 7, '-sua': 2, 'sua-': 2, 'su-ă': 1, 'şu-a': 1})
la: 0 0 Counter({'la': 7530287, 'l-a': 173045, 'lă': 815, 'la-': 507, '-la': 277, 'lâ': 142, 'lă-': 51, 'l-a-': 37, 'l-ă': 15, 'l-â': 8, '-l-a': 6, 'lâ-': 3, '-lâ': 2, '--la': 1})
natiunile: 0 0 Counter({'naţiunile': 3478, 'natiunile': 1})
unite: 0 0 Counter({'unite': 28812, 'u-nite': 3, 'unite-': 2, 'uniţe': 1, '-unite': 1})
nikky: 0 0 Counter({'nikky': 8})
haley: 0 0 Counter({'haley': 436})
a: 0 0 Counter({'a': 7144062, 'ă': 25589, 'a-': 9259, '-a': 1644, 'â': 614, '-ă': 312, 'ă-': 37, '-â': 22, 'â-': 16, '-a-': 10, '-ă-': 9, '-â-': 4, '--a': 1})
declarat: 0 0 Counter({'declarat': 16054, 'declarat-': 11, 'declaraţ': 2, 'de-clarat': 1, 'declarât': 1, 'decla-rat': 1})
miercuri: 0 0 Counter({'miercuri': 5547, 'mier-curi': 2})
ca: 0 0 Counter({'că': 5741492, 'ca': 3108117, 'c-a': 7643, 'că-': 4257,

In [181]:
import sys
for var, obj  in [ ( name, ob) for name, ob in locals().items() if name[0]!="_"]:
#     print(var, sys.getsizeof(obj), type(obj))
    pass